<!--<badge>--><a href="https://colab.research.google.com/github/huggingface/workshops/blob/main/europython-2022/03-gradio-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# Creating a Transformers demo with Gradio

## References

* https://huggingface.co/blog/gradio-spaces
* https://huggingface.co/blog/gradio

In [ ]:
# !pip install transformers gradio sentencepiece

## Example 1: Using the Transformers pipeline

In [1]:
import gradio as gr
from transformers import pipeline

In [2]:
pipe = pipeline("text-classification", model="Rocketknight1/europython-imdb")

2022-07-10 16:58:58.104883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 16:58:58.108973: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 16:58:58.109835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 16:58:58.162488: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
pipe("This was the most boring 94 minutes of my life.")

[{'label': 'Negative', 'score': 0.9972227811813354}]

In [4]:
label2emoji = {"Negative": "💩", "Positive": "😻"}

def predict(text):
    preds = pipe(text)[0]
    return label2emoji[preds["label"]], round(preds["score"], 5)

predict("I loved this movie!")

('😻', 0.98438)

In [5]:
gradio_ui = gr.Interface(
    fn=predict,
    title="Review analysis",
    description="Enter some review text and check what the model predicts for its sentiment.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["I loved it, best movie ever!"], ["Turgid and preposterous, almost unwatchable."]
    ],
)

gradio_ui.launch(debug=True)

/home/matt/miniconda3/envs/tensorflow28/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/matt/miniconda3/envs/tensorflow28/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/matt/miniconda3/envs/tensorflow28/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f79f4618100>, 'http://127.0.0.1:7860/', None)

## Example 2: Using the Inference API from the Hugging Face Hub

In [6]:
from huggingface_hub import InferenceApi

text = "I loved it, best movie ever!"
inference = InferenceApi("Rocketknight1/europython-imdb")
preds = inference(inputs=text)
preds[0]

[{'label': 'Positive', 'score': 0.9859941005706787},
 {'label': 'Negative', 'score': 0.0140059320256114}]

In [7]:
sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True) 
sorted_preds

[{'label': 'Positive', 'score': 0.9859941005706787},
 {'label': 'Negative', 'score': 0.0140059320256114}]

In [10]:
def inference_predict(text):
    inference = InferenceApi("Rocketknight1/europython-imdb")
    preds = inference(inputs=text)
    sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True)[0]
    return label2emoji[sorted_preds["label"]], round(sorted_preds["score"], 5)

In [11]:
inference_predict(text)

('😻', 0.98599)

In [12]:
gradio_ui = gr.Interface.load(
    name="Rocketknight1/europython-imdb",
    src="huggingface",
    fn=inference_predict,
    title="Review analysis",
    description="Enter some text and check if model detects it's star rating.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["I loved it, best movie ever!"], ["Turgid and preposterous, almost unwatchable."]
    ],
)

gradio_ui.launch(debug=True)

/home/matt/miniconda3/envs/tensorflow28/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/matt/miniconda3/envs/tensorflow28/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/matt/miniconda3/envs/tensorflow28/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Fetching model from: https://huggingface.co/Rocketknight1/europython-imdb
Running on local URL:  http://127.0.0.1:7861/

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f79f46c7d30>, 'http://127.0.0.1:7861/', None)